In [ ]:
# from pprint import pprint.pprint as pprint
import pprint

Setup
-----------

Run the following series of cells to initialize the required dependancies and 
load the dataset

In [ ]:
import pandas as pd
# ENTER YOUR DATSET AS A PANDAS DATAFRAME NAMED 'df' HERE

In [ ]:
#  This would be where you choose what document size you want
rows = df["Body"][200:500]
rows = rows.array.tolist()

Hugging Face Semantic Search
---------

In [ ]:
pip install -U sentence-transformers

In [ ]:
# Upload the data and replace the path. Then run the following 2 cells to get the tagged email data for emails 200-500
!unzip /content/97e76ea3-797e-4955-8973-20b8f965f924.zip

Archive:  /content/97e76ea3-797e-4955-8973-20b8f965f924.zip
  inflating: admin.jsonl             


In [ ]:
import json
# This was run in google colab, you may need to adjust the path depending on
# your environment
filename = "/content/admin.jsonl"
with open(filename) as f:
  lines = f.readlines()

labled_data= []
for s in lines:
  result = json.loads(s)
  labled_data.append(result)

# some of the emails may have been put out of order so this fixes that
labled_data.sort(key = lambda x: x['id'])

In [ ]:
# this is the form for the output of the function
# [[query0,[result0,result1,result2]],[query1,[result0,result1,result2]]...]
def run_queries(model_name, queries, rows, similarity,num_results):
  # initializes the array that will contain the return value of the function
  model_storage = []
  for i in range(len(queries)):
    model_storage.append([queries[i],[]])

  model = SentenceTransformer(model_name)
  doc_emb = model.encode(rows)

  for i in range(len(queries)):
    query_emb = model.encode(queries[i])

    # chooses the similarity search. If you want, you can add others
    if (similarity == 'dot'):
      scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
    elif (similarity == 'cos'):
      scores = util.cos_sim(query_emb, doc_emb)[0].cpu().tolist()
    else:
      raise ValueError('invalid similarity! Supported arguments: dot, cos')
    
    # selects the num_results number best results for the query by similarity
    doc_score_pairs = list(zip(rows, scores))
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    model_storage[i][1] = doc_score_pairs[0:num_results]

  return model_storage

In [ ]:
# model results is the list produced by run_queries
# rows is the original data onverted to a list
# labled_data is the docanno labled data
def compute_model_score(model_results, rows, labeled_data):
  score = 0
  for x in model_results:  
    # converts between the Doccano document label and the desired semantic search query 
    if x[0] == "public releases":
      query = "public releases (press and state)"
    else:
      query = x[0]
    
    for y in x[1]:
      # if there were no lables for the data automatically fail the test
      if len(labled_data[rows.index(y[0])]["label"]) == 0:
        score-=1
      else:
        # determines if the retrieved document is relevant for the query
        if (query in labeled_data[rows.index(y[0])]["label"][0]):
          score+= 1
        else:
          score-=1 
          
  return score    


In [ ]:
# print(x[1][0][1][0][0]) # <-- gives actual message
#   print(x[1][0][0]) # <-- prints label

In [ ]:
# This is where we see which model performed best on the data
# Runs all of the models on the documents and sees which ones produce emails
# that have tags relevant to the queries. 
# This cell returns the score of each model for selecting relevant emails for queries.
# Note that higher numbers (less negative) are better than smaller ones
from sentence_transformers import SentenceTransformer, util
import pprint

# intimidation is omitted as there is only one instance of it in the dataset
queries = ["taking responsibility", 
          #  "intimidation", 
           "bad judgement because of groupthink", 
           "confusion", 
           "opportunism",
           "public releases"]


# feel free to add other models here for testing
dot_models = ["sentence-transformers/all-mpnet-base-v2",
              "sentence-transformers/all-MiniLM-L12-v2",
              "sentence-transformers/msmarco-bert-base-dot-v5"]

cos_models = ["sentence-transformers/msmarco-distilbert-cos-v5",
              "sentence-transformers/msmarco-distilbert-base-v4"]


# this runs the search queries. It can take a really long time
dot_model_results = []
for x in dot_models:
  dot_model_results.append([x,run_queries(x,queries,rows,'dot',5)])

cos_model_results = []
for x in cos_models:
  cos_model_results.append([x,run_queries(x,queries,rows,'cos',5)])


# scores models based on their searches and outputs the result
all_model_scores = []

for x in dot_model_results:
  all_model_scores.append((x[0], compute_model_score(x[1],rows,labled_data)))

for x in cos_model_results:
  all_model_scores.append((x[0], compute_model_score(x[1],rows,labled_data)))

pprint.pprint(all_model_scores)



When num_results=3 these are the scores
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[('sentence-transformers/all-mpnet-base-v2', -15),
 ('sentence-transformers/all-MiniLM-L12-v2', -13),
 ('sentence-transformers/msmarco-bert-base-dot-v5', -9),
 ('sentence-transformers/msmarco-distilbert-cos-v5', -11),
 ('sentence-transformers/msmarco-distilbert-base-v4', -11)]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 when num_results=5 these are the scores:
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 [('sentence-transformers/all-mpnet-base-v2', -19),
 ('sentence-transformers/all-MiniLM-L12-v2', -21),
 ('sentence-transformers/msmarco-bert-base-dot-v5', -15),
 ('sentence-transformers/msmarco-distilbert-cos-v5', -21),
 ('sentence-transformers/msmarco-distilbert-base-v4', -21)]
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 'sentence-transformers/msmarco-bert-base-dot-v5' performs best in both trials

In [ ]:
# This is the UI that calls all the other functions and does the actual searches

# feel free to add other queries to this list. However, you should make sure
# that those are supported in the Doccano tags. 
queries = ["taking responsibility", 
           "intimidation", 
           "bad judgement because of groupthink", 
           "confusion", 
           "opportunism",
           "public releases"]


# choose which documents you want to search on
rows2 = df["Body"][:]
rows2 = rows2.array.tolist()

pprint.pprint(run_queries('sentence-transformers/msmarco-bert-base-dot-v5',queries,rows2,'dot',3))